In [24]:
%matplotlib inline
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.pyplot import figure
import plotly.graph_objects as go

In [2]:
def N_(x_,y_,sigma_x,sigma_y,corr):
    mean = [x_,y_]
    sigma_xy = corr * sigma_x * sigma_y
    cov = [[sigma_x**2,sigma_xy],[sigma_xy,sigma_y**2]]
    model = st.multivariate_normal(mean=mean,cov=cov)
    def func(size):
        return model.rvs(size)
    return func

In [3]:
def r(c):
    mx = 0
    for i in range(len(c)):
        mx += c[i][0]
    mx /= len(c)

    my = 0
    for i in range(len(c)):
        my += c[i][1]
    my /= len(c)

    K = 0
    for i in range(len(c)):
        K += (c[i][0] - mx) * (c[i][1] - my)
    K /= len(c)


    dx = 0
    for i in range(len(c)):
        dx += (c[i][0] - mx)**2
    dx/= len(c)

    dy = 0
    for i in range(len(c)):
        dy += (c[i][1] - my)**2
    dy/= len(c)

    r = K/(dx*dy)**0.5

    return r

In [4]:
def rq(array):
    n1 = 0
    n2 = 0
    n3 = 0
    n4 = 0
    
    x = array[:,0].copy()
    y = array[:,1].copy()
    
    x.sort()
    y.sort()
    
    if(len(x) % 2 == 0):
        x_med = (x[len(x)//2] + x[len(x)//2 - 1])/2
    else:
        x_med = x[len(x)//2]
        
    if(len(y) % 2 == 0):
        y_med = (y[len(y)//2] + y[len(y)//2 - 1])/2
    else:
        y_med = y[len(y)//2]
 
    for i in range(len(array)):
        if (array[i][0] >= x_med) and (array[i][1] >= y_med):
            n1 += 1
        if (array[i][0] < x_med) and (array[i][1] >= y_med):
            n2 += 1
        if (array[i][0] < x_med) and (array[i][1] < y_med):
            n3 += 1
        if (array[i][0] >= x_med) and (array[i][1] < y_med):
            n4 += 1
    return ((n1 + n3) - (n2 + n4))/len(array)

In [5]:
def rS(array_2d):
    mx = (len(array_2d) + 1)/2
    my = (len(array_2d) + 1)/2
    
    x = array_2d[:,0].copy()
    y = array_2d[:,1].copy()
    
    x.sort()
    y.sort()
    
    xMap = dict()
    yMap = dict()
    
    for i in range(len(x)):
        xMap[x[i]] = i + 1
    
    for i in range(len(y)):
        yMap[y[i]] = i + 1

    K = 0
    for i in range(len(array_2d)):
        K += (xMap[array_2d[i][0]] - mx) * (yMap[array_2d[i][1]] - my)
    K /= len(array_2d)


    dx = 0
    for i in range(len(array_2d)):
        dx += (xMap[array_2d[i][0]] - mx)**2
    dx/= len(array_2d)

    dy = 0
    for i in range(len(array_2d)):
        dy += (yMap[array_2d[i][1]] - my)**2
    dy/= len(array_2d)

    val = K/(dx*dy)**0.5

    return val

In [42]:
def getMean1(array):
    sm = 0
    for val in array:
        sm += val
    sm /= len(array)
    return sm

def getMean2(array):
    sm = 0
    for val in array:
        sm += val**2
    sm /= len(array)
    return sm

def getDis(array):
    sm = 0
    mn = getMean(array)
    for i in array:
        sm += (i - mn) ** 2
    return sm/ len(array)

def getMetricsRepeat(func,size = 100, iterations = 1000):
    array = func(size)
    ar = [[rS(array),rq(array),r(array)]]
    for i in range(999):
        array = func(size)
        ar = np.concatenate(([[r(array),rS(array),rq(array)]],ar),axis = 0)
    disp = []
    mean = []
    mean2 = []
    for i in range(len(ar[0])):
        mean.append(getMean(ar[:,i]))
        disp.append(round(getDis(ar[:,i]),2))
        mean2.append(getMean2(ar[:,i]))
        
    return {'mean':mean,
            'disp' : disp,
            'mean2' : mean2
           }

def Draw(func,size):
    d = getMetricsRepeat(func,size)
    vals = ['r','rS','rQ']
    col = [['mean1'] + ['mean2'] + ['disp']]
    for i in range(0,len(d['mean'])):
        col.append([d['mean'][i]] + [d['mean2'][i]] + [d['disp'][i]] )
    fig = go.Figure(data=[go.Table(
        header=dict(values=['n = ' +str(size)] + vals,
                    line_color='darkslategray',
                    fill_color='lightskyblue',
                    align='left',
         ),
        cells=dict(values=col, # 2nd column
                   line_color='darkslategray',
                   fill_color='lightcyan',
                   align='left',
                  format = ["",".3f"]))
    ])

    fig.update_layout(width=1000, height=300)
    fig.show()

In [46]:
def func(size = 20):
    f1 = N_(0,0,1,1,0.9)
    f2 = N_(0,0,10,10,-0.9)
    return 0.9*f1(size) + 0.1 * f2(size)

In [53]:
f = N_(0,0,1,1,0.9)
Draw(f,20)
Draw(f,60)
Draw(f,100)

In [54]:
Draw(func,20)
Draw(func,60)
Draw(func,100)